In [1]:
import os
import bs4
import json
import requests

In [2]:
# cada vez que hacemos una toca al inali.cob.mx, recibimos una alarma así:
# InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised.
# creo q los certificados de INALI se han caducado
# por no ver la alarma cada vez que agarramos un vínculo de INALI, la deshabilitamos aquí

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
# esto es el URL donde se encuentra todos los vínculos de las agrupaciones lingüísticas
base_url = 'https://www.inali.gob.mx/clin-inali/'

In [4]:
# ver aquí más detalle sobre "verify" y certificates:
# https://stackoverflow.com/questions/28667684/python-requests-getting-sslerror
r = requests.get(base_url, verify=False)

In [5]:
# primero trabajamos en un ejemplo específico; luego lo generalizamos
# empezamos en esta página (l_nahuatl.html) para grabar la información de Familia Lingüística
nahuarl = 'https://www.inali.gob.mx/clin-inali/html/l_nahuatl.html'
r = requests.get(nahuarl, verify=False)
soup = bs4.BeautifulSoup(r.text, 'html.parser')
agrup_ling = soup.body.h4.contents[0]    # Agrupación lingüística
familia_ling = soup.body.h4.contents[-1]  # Familia lingüística
print(agrup_ling, familia_ling)

# sacamos solo los datos que nos interesan
agrup_ling = agrup_ling.split(':')[-1].strip()
familia_ling = familia_ling.split(':')[-1].strip()
print(agrup_ling, familia_ling)

Agrupación lingüística: náhuatl Familia lingüística: Yuto-nahua
náhuatl Yuto-nahua


In [6]:
# ya tenemos los datos que necesitamos de esta página,
# seguimos a un nivel más bajo, las agrupaciónes lingüísticas
soup.find_all('a')  # para verificar si todos los vínculos están dirigidos a la misma página

[<a class="mlink" href="v_nahuatl.html#1">náhuatl de la Sierra, noreste de Puebla</a>,
 <a class="mlink" href="v_nahuatl.html#2">náhuatl del noroeste central</a>,
 <a class="mlink" href="v_nahuatl.html#3">náhuatl del Istmo</a>,
 <a class="mlink" href="v_nahuatl.html#4">náhuatl de la Huasteca veracruzana</a>,
 <a class="mlink" href="v_nahuatl.html#5">náhuatl de la Huasteca potosina</a>,
 <a class="mlink" href="v_nahuatl.html#6">náhuatl de Oaxaca</a>,
 <a class="mlink" href="v_nahuatl.html#7">náhuatl de la Sierra negra, sur</a>,
 <a class="mlink" href="v_nahuatl.html#8">náhuatl de la Sierra negra, norte</a>,
 <a class="mlink" href="v_nahuatl.html#9">náhuatl central de Veracruz</a>,
 <a class="mlink" href="v_nahuatl.html#10">náhuatl de la Sierra oeste de Puebla</a>,
 <a class="mlink" href="v_nahuatl.html#11">náhuatl alto del norte de Puebla</a>,
 <a class="mlink" href="v_nahuatl.html#12">náhuatl del Istmo bajo</a>,
 <a class="mlink" href="v_nahuatl.html#13">náhuatl del centro de Puebla</a

In [7]:
# comparamos la página entera con las piezas de ella del estilo <a>v_nahuatl#1/>
# para ver si este método sea más fácil/eficiente
vincs = soup.find_all('a')
vincs[0].attrs['href']
agrup_url_pieza = ''.join([base_url, 'html/', vincs[0].attrs['href']])
agrup_url_pieza

'https://www.inali.gob.mx/clin-inali/html/v_nahuatl.html#1'

In [8]:
# construimos el URL de una agrupación lingüística entera y sacamos el contenido
agrup_url = ''.join([base_url, 'html/', soup.body.a.attrs['href']])
r = requests.get(agrup_url, verify=False)
sopa = bs4.BeautifulSoup(r.text, 'html.parser')

In [9]:
# ya veremos que son los mismos datos: que el estile del vínculo'v_nahuatl.html#1'
# sólo es para que el usuario del sitio pueda navigar más facilmente,
# o sea, sirva para llevarnos a una part éspecifica de la página completa de nauhuatl
# pero no es relevante a nuestra busqueda iterable
r = requests.get(agrup_url_pieza, verify=False)
sopa_nahuatl_01 = bs4.BeautifulSoup(r.text, 'html.parser')
sopa_nahuatl_01 == sopa

True

In [10]:
# los datos éspecificos que buscamos aparacen 
# entre los "rows" <tr></tr> tags que hay en una página
# por ejemplo la página de nahuatl tiene 31 rows
# (la primera row (aquí representado como all_trs[0]) no nos importa porque no tiene un table)
all_trs = sopa.find_all('tr')
print(len(all_trs))

31


In [35]:
all_trs[2]

<tr>
<td><a name="2"></a>
<p>mexi’catl
		<br/><span>[meʃiʔkaλ]</span></p>
<p>mexicano (del noroeste central)
		<br/><span>[mexikano]</span></p>
<p>maseual tla’tol
		<br/><span>[masewal λaʔtol]</span></p>
		&lt;náhuatl del noroeste central&gt;
                </td>
<td>
                    HIDALGO: <b>Acaxochitlán</b>: Acaxochitlán, Alpinagua, Apapaxtla (Altamira), Apopoalco, Atezcapa, Barrio Cuautenco, Barrio el Puente, Barrio Tlatempa, Barrio Tlatzintla, Barrio Verde, Buenavista (Necaxanco), Coyametepec, Cuaunepantla, Chimalapa, Ejido Techachalco, Ejido de Tepepa (Barrio Santa Félix), Ejido de Tlatzintla, El Bado, El Crucero, El Tejocotal, El Valle, La Boveda, La Mesa, Los Reyes, Montemar, Necaxanco, Nuevo San Juan, Ocotengo, Ojo de Agua las Palomas, Parada 170, Paredones, San Fernando, San Francisco Atotonilco, San Juan, San Mateo, San Miguel del Resgate, San Pedro Tlachichilco, Santa Ana Tzacuala, Santa Catarina, Techachalco, Tepepa (Santiago Tepepa), Tlacpac, Tlamimilolpa, Toxtla, 

In [36]:
# más específicamente, los datos del los lenguajes quedan dentro
# de los tables que así mismos están dentro de un "row"
# es decir, encontramos los datos que nos interesa entre los tags <td></td>
all_tds = all_trs[2].find_all('td')
all_tds

[<td><a name="2"></a>
 <p>mexi’catl
 		<br/><span>[meʃiʔkaλ]</span></p>
 <p>mexicano (del noroeste central)
 		<br/><span>[mexikano]</span></p>
 <p>maseual tla’tol
 		<br/><span>[masewal λaʔtol]</span></p>
 		&lt;náhuatl del noroeste central&gt;
                 </td>, <td>
                     HIDALGO: <b>Acaxochitlán</b>: Acaxochitlán, Alpinagua, Apapaxtla (Altamira), Apopoalco, Atezcapa, Barrio Cuautenco, Barrio el Puente, Barrio Tlatempa, Barrio Tlatzintla, Barrio Verde, Buenavista (Necaxanco), Coyametepec, Cuaunepantla, Chimalapa, Ejido Techachalco, Ejido de Tepepa (Barrio Santa Félix), Ejido de Tlatzintla, El Bado, El Crucero, El Tejocotal, El Valle, La Boveda, La Mesa, Los Reyes, Montemar, Necaxanco, Nuevo San Juan, Ocotengo, Ojo de Agua las Palomas, Parada 170, Paredones, San Fernando, San Francisco Atotonilco, San Juan, San Mateo, San Miguel del Resgate, San Pedro Tlachichilco, Santa Ana Tzacuala, Santa Catarina, Techachalco, Tepepa (Santiago Tepepa), Tlacpac, Tlamimilolpa, To

### sabemos que lo que está dentro del primer "table" (td tags) son, en primer lugar, las autodenominaciones, y después el variante
##### parace que el variante no queda entre "paragraph HTML tags" mientras las autodenominaciones sí. hará que confirmar esto con otros ejemplos, pero si eso sea lo verdad, será más fácil distinguir entre los dos.

In [37]:
all_tds[0].contents

[<a name="2"></a>, '\n', <p>mexi’catl
 		<br/><span>[meʃiʔkaλ]</span></p>, '\n', <p>mexicano (del noroeste central)
 		<br/><span>[mexikano]</span></p>, '\n', <p>maseual tla’tol
 		<br/><span>[masewal λaʔtol]</span></p>, '\n\t\t<náhuatl del noroeste central>\n                ']

In [38]:
all_tds[0].text.strip().split('\t\t')

['mexi’catl\n',
 '[meʃiʔkaλ]\nmexicano (del noroeste central)\n',
 '[mexikano]\nmaseual tla’tol\n',
 '[masewal λaʔtol]\n',
 '<náhuatl del noroeste central>']

In [39]:
# aquí tenemos el VARIANTE
variante = all_tds[0].contents[-1].strip()
variante

'<náhuatl del noroeste central>'

In [40]:
# aquí tenemos los AUTODENOMINACIÓNes
pstrings = []
for p in all_tds[0].find_all('p'):
    for pstr in p.strings:
        pstrings.append(pstr.strip())
print('autodenominaciones extraidos del table: {}'.format(pstrings))

# NOTA: este método de extraer las autodenominaciones presupone que siempre hay dos
# maneras de describir las autods: primero como string y luego lo mismo de otra forma en brackets.
# entonces lo separamos con un newline '\n' como aparece en la página de INALI
autodes = []
for i in range(len(pstrings)):
    if (i+1) % 2 == 0:
        autode = '/n'.join([pstrings[i-1], pstrings[i]])
        autodes.append(autode)
autodes

autodenominaciones extraidos del table: ['mexi’catl', '[meʃiʔkaλ]', 'mexicano (del noroeste central)', '[mexikano]', 'maseual tla’tol', '[masewal λaʔtol]']


['mexi’catl/n[meʃiʔkaλ]',
 'mexicano (del noroeste central)/n[mexikano]',
 'maseual tla’tol/n[masewal λaʔtol]']

### seguimos con los datos geográficos

In [17]:
for td in all_tds:
    for string in td.strings:
        print(string.strip())


mexicano tlajtol
[mexikano λahtol]

nauta
[nawta]
<náhuatl de la Sierra, noreste de Puebla>
PUEBLA:
Atempan
: Apatauyan, Atempan, Atzalán, Cala Norte, Cala Sur, El Cuatro, Huexoteno, Las Canoas, Las Delicias, Meyuco, Potrero Nuevo, San Ambrosio, San Nicolás (Animazco), Tacopan, Tanhuixco, Tezhuatepec, Tezompan.
Ayotoxco de Guerrero
: Acaltzontán, Agua Bonita, Arenal, Ayotoxco de Guerrero, Buena Vista, Casa Quemada, Colozapan, Copales, Cuauhtémoc (Tlacuilolostoc), El Roble, Flores Villar, Gachupinate (Rancho Nuevo), La Lagunilla, La Manigua, La Quebradora, La Unión, Las Ceibas, Las Cibeles, Las Cruces, Los Aguacates, Los Mangos, Malintepec, Nanacatepec, Plan de la Palma, San Antonio Metzonapa, San Francisco Atecacalax, Santa Cecilia, Santa Elena, Trapiche Viejo, Uruapan, Vega Redonda, Xaltorre.
Cuautempan
: Ahuatlán, Cerro Verde (Ejecatepeco), Hueytentán, Ixtolco de Morelos, Papalotla, San Esteban Cuautempan, San José Río Bravo, Taltzontipan, Tecapagco, Tenepanigia (Santa Elena), Tepiz

### ya vimos que el primer table fue el que tiene la información sobre la autodenominacines y el variante, así que el segundo table deben corresponder a un estado, sus municipios, y luego sus localidades donde se hablan este variante. Si hay más que un estado en donde se hable este variante, estan separados en la pagina con un "linebreak"

In [41]:
len(all_tds)

2

In [43]:
gdatos = all_tds[1]
gdatos

<td>
                    HIDALGO: <b>Acaxochitlán</b>: Acaxochitlán, Alpinagua, Apapaxtla (Altamira), Apopoalco, Atezcapa, Barrio Cuautenco, Barrio el Puente, Barrio Tlatempa, Barrio Tlatzintla, Barrio Verde, Buenavista (Necaxanco), Coyametepec, Cuaunepantla, Chimalapa, Ejido Techachalco, Ejido de Tepepa (Barrio Santa Félix), Ejido de Tlatzintla, El Bado, El Crucero, El Tejocotal, El Valle, La Boveda, La Mesa, Los Reyes, Montemar, Necaxanco, Nuevo San Juan, Ocotengo, Ojo de Agua las Palomas, Parada 170, Paredones, San Fernando, San Francisco Atotonilco, San Juan, San Mateo, San Miguel del Resgate, San Pedro Tlachichilco, Santa Ana Tzacuala, Santa Catarina, Techachalco, Tepepa (Santiago Tepepa), Tlacpac, Tlamimilolpa, Toxtla, Venta Quemada, Yemila, Zacacuautla, Zotictla.

<br/><br/>
PUEBLA: <b>Chiconcuautla</b>: Acalama, Amola, Axocopactla, Azacatla, Benito Juárez, Cosamaloapan, Cuetzalingo, Chiconcuautla, Huautla, Huixtlacuatla, Itlantixcali, Ixtaczoquitla, Macuilacatla, Mimitla, Ojo d

In [57]:
for thing in gdatos:
    print(thing)
    print(type(thing))
    if thing.__class__ == bs4.element.Tag:
        print(thing.decode() == '<br/>')


                    HIDALGO: 
<class 'bs4.element.NavigableString'>
<b>Acaxochitlán</b>
<class 'bs4.element.Tag'>
False
: Acaxochitlán, Alpinagua, Apapaxtla (Altamira), Apopoalco, Atezcapa, Barrio Cuautenco, Barrio el Puente, Barrio Tlatempa, Barrio Tlatzintla, Barrio Verde, Buenavista (Necaxanco), Coyametepec, Cuaunepantla, Chimalapa, Ejido Techachalco, Ejido de Tepepa (Barrio Santa Félix), Ejido de Tlatzintla, El Bado, El Crucero, El Tejocotal, El Valle, La Boveda, La Mesa, Los Reyes, Montemar, Necaxanco, Nuevo San Juan, Ocotengo, Ojo de Agua las Palomas, Parada 170, Paredones, San Fernando, San Francisco Atotonilco, San Juan, San Mateo, San Miguel del Resgate, San Pedro Tlachichilco, Santa Ana Tzacuala, Santa Catarina, Techachalco, Tepepa (Santiago Tepepa), Tlacpac, Tlamimilolpa, Toxtla, Venta Quemada, Yemila, Zacacuautla, Zotictla.


<class 'bs4.element.NavigableString'>
<br/>
<class 'bs4.element.Tag'>
True
<br/>
<class 'bs4.element.Tag'>
True

PUEBLA: 
<class 'bs4.element.Navigab

In [22]:
i = 0
for cosa in gdatos:
    print(type(cosa))
    print(dir(cosa))
    i += 1
    if i > 2:
        break

<class 'bs4.element.NavigableString'>
['PREFIX', 'SUFFIX', '__add__', '__class__', '__contains__', '__copy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_find_all', '_find_one', '_is_xml', '_lastRecursiveChild', '_last_descendant', 'append', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'extend', 'extract', 'fetchNextSiblings', 'fetchParents', 'fetchPrevious', 'fetchPreviousSiblings', 'find', 'findAllNext', 'findAllPrevious', 'findNext', 'findNextSibling', 'findNextSiblings', 'findParent', 'findParents', 'findPrevious', 'findPreviousSibling', 'findPrev

In [76]:
def parse_datos_geos(datos_geos):
    '''@datos_geo debe ser un table de HTML'''
    repr_geo = {}
    
    for parte in datos_geos:
        # primero identificamos el estado, que siempre está en mayúsculas
        # esta parte del table is of class 'bs4.element.NavigableString'
        if parte.__class__ == bs4.element.NavigableString:
            if parte.isupper():
                estado = parte.strip(' :\t\r\n')
                repr_geo[estado] = {}
                continue
                
        # luego, si el parte tiene los bold HTML tags "<br></br>"
        # es otro tipo de bs4 object of class 'bs4.element.Tag'
        if parte.__class__ == bs4.element.Tag and parte.decode() != '<br/>':
            municipio = parte.text
            repr_geo[estado][municipio] = []
            
        # finalmente, los localidades también son de la class 'bs4.element.NavigableString'
        # si ya tenemos el estado, sabemos que esta parte describe localidades
        if parte.__class__ == bs4.element.NavigableString and repr_geo[estado]:
            localidades = parte.strip(' :\n')
            repr_geo[estado][municipio].extend([localidades])
            
        # si hay multiples estados en los que se hablan la misma variante
        # van a ser separados por "linebreaks". entonces usamos el linebreak
        # como un flag para identificar que hay que construir otro repr_geo
        if parte.__class__ == bs4.element.Tag and parte.decode() == '<br/>':
            yield repr_geo
            repr_geo = {}
    
    # yield el último 
    yield repr_geo
        
    # return la representación
    # return repr_geo

repr_geo = [geo for geo in parse_datos_geos(gdatos)]
repr_geo

[{'HIDALGO': {'Acaxochitlán': ['Acaxochitlán, Alpinagua, Apapaxtla (Altamira), Apopoalco, Atezcapa, Barrio Cuautenco, Barrio el Puente, Barrio Tlatempa, Barrio Tlatzintla, Barrio Verde, Buenavista (Necaxanco), Coyametepec, Cuaunepantla, Chimalapa, Ejido Techachalco, Ejido de Tepepa (Barrio Santa Félix), Ejido de Tlatzintla, El Bado, El Crucero, El Tejocotal, El Valle, La Boveda, La Mesa, Los Reyes, Montemar, Necaxanco, Nuevo San Juan, Ocotengo, Ojo de Agua las Palomas, Parada 170, Paredones, San Fernando, San Francisco Atotonilco, San Juan, San Mateo, San Miguel del Resgate, San Pedro Tlachichilco, Santa Ana Tzacuala, Santa Catarina, Techachalco, Tepepa (Santiago Tepepa), Tlacpac, Tlamimilolpa, Toxtla, Venta Quemada, Yemila, Zacacuautla, Zotictla.']}},
 {}]

### ya tenemos todos los datos que queremos para hacer un ejemplo

In [24]:
agrup_ling

'náhuatl'

In [25]:
familia_ling

'Yuto-nahua'

In [26]:
autodes

['mexicano tlajtol/n[mexikano λahtol]', 'nauta/n[nawta]']

In [27]:
variante

'<náhuatl de la Sierra, noreste de Puebla>'

In [28]:
repr_geo

{'PUEBLA': {'Atempan': ['Apatauyan, Atempan, Atzalán, Cala Norte, Cala Sur, El Cuatro, Huexoteno, Las Canoas, Las Delicias, Meyuco, Potrero Nuevo, San Ambrosio, San Nicolás (Animazco), Tacopan, Tanhuixco, Tezhuatepec, Tezompan.'],
  'Ayotoxco de Guerrero': ['Acaltzontán, Agua Bonita, Arenal, Ayotoxco de Guerrero, Buena Vista, Casa Quemada, Colozapan, Copales, Cuauhtémoc (Tlacuilolostoc), El Roble, Flores Villar, Gachupinate (Rancho Nuevo), La Lagunilla, La Manigua, La Quebradora, La Unión, Las Ceibas, Las Cibeles, Las Cruces, Los Aguacates, Los Mangos, Malintepec, Nanacatepec, Plan de la Palma, San Antonio Metzonapa, San Francisco Atecacalax, Santa Cecilia, Santa Elena, Trapiche Viejo, Uruapan, Vega Redonda, Xaltorre.'],
  'Cuautempan': ['Ahuatlán, Cerro Verde (Ejecatepeco), Hueytentán, Ixtolco de Morelos, Papalotla, San Esteban Cuautempan, San José Río Bravo, Taltzontipan, Tecapagco, Tenepanigia (Santa Elena), Tepizila (Primera Sección), Texamanila, Texocotitán, Tlapacholoya, Totocuau

### podemos representar los datos con un modelo de JSON:

In [29]:
datos_de_un_variante = {
    'agrupacion_linguïstica': '',
    'familia_linguïstica': '',
    'autodenominaciones': [],
    'variante': '',
    'representación_geográfica': {}
}

In [30]:
# usando nuestro ejemplo de nahuatl llegamos al siguiente
datos_de_nahuatl_de_puebla = {
    'agrupacion_linguïstica': agrup_ling,
    'familia_linguïstica': familia_ling,
    'autodenominaciones': autodes,
    'variante': variante,
    'representación_geográfica': repr_geo
}

In [31]:
datos_de_nahuatl_de_puebla

{'agrupacion_linguïstica': 'náhuatl',
 'familia_linguïstica': 'Yuto-nahua',
 'autodenominaciones': ['mexicano tlajtol/n[mexikano λahtol]',
  'nauta/n[nawta]'],
 'variante': '<náhuatl de la Sierra, noreste de Puebla>',
 'representación_geográfica': {'PUEBLA': {'Atempan': ['Apatauyan, Atempan, Atzalán, Cala Norte, Cala Sur, El Cuatro, Huexoteno, Las Canoas, Las Delicias, Meyuco, Potrero Nuevo, San Ambrosio, San Nicolás (Animazco), Tacopan, Tanhuixco, Tezhuatepec, Tezompan.'],
   'Ayotoxco de Guerrero': ['Acaltzontán, Agua Bonita, Arenal, Ayotoxco de Guerrero, Buena Vista, Casa Quemada, Colozapan, Copales, Cuauhtémoc (Tlacuilolostoc), El Roble, Flores Villar, Gachupinate (Rancho Nuevo), La Lagunilla, La Manigua, La Quebradora, La Unión, Las Ceibas, Las Cibeles, Las Cruces, Los Aguacates, Los Mangos, Malintepec, Nanacatepec, Plan de la Palma, San Antonio Metzonapa, San Francisco Atecacalax, Santa Cecilia, Santa Elena, Trapiche Viejo, Uruapan, Vega Redonda, Xaltorre.'],
   'Cuautempan': ['A

In [32]:
# output nuestro trabajo para compartir
def output_variante_json(datos):
    '''construir un filename del nombre de variante y escribir los datos a un archivo de JSON'''
    variante_split = datos['variante'].strip('<>').split(' ')
    datos_file_ending = ('_').join([vsplit.strip(',') for vsplit in variante_split])
    outfile = os.path.join('output', 'datos_de_{}.json'.format(datos_file_ending))
    with open(outfile, 'w') as outf:
        outf.write(json.dumps(datos))
        
output_variante_json(datos_de_nahuatl_de_puebla)